### *This file allows to reproduce FigS2*

# **Useful packages and functions**

In [ ]:
using DifferentialEquations, Plots, Plots.PlotMeasures, LaTeXStrings, Random, Dierckx, DelimitedFiles
using Interpolations
include("STG_kinetics.jl") # Loading of STG kinetics of gating variables
include("STG_models.jl") # Loading of STG model
include("STG_utils.jl") # Loading of some utils functions
include("STG_gs_derivatives.jl") # Loading of X_inf derivatives
include("STG_DIC.jl") # Loading of the DIC and compensation algorithm
include("STG_neuromodulation.jl") # Loading of the neuromodulation cells functions
include("STG_large_simulations.jl"); # Loading of the neuromodulation cells functions

# **Global variables**

In [ ]:
# Definition of simulation time (in ms)
const Tfinal = 300000
const tspan  = (0.0, Tfinal)

# Definition of reversal potential values (in mV) and membrane capacitance
const VNa   = 50. # Sodium reversal potential
const VK    = -80. # Potassium reversal potential
const VCa   = 80. # Calcium reversal potential
const VH    = -20. # Reversal potential for the H-current (permeable to both sodium and potassium ions)
const Vleak = -50. # Reversal potential of leak channels
const C     = 1. # Membrane capacitance

# Definition of voltage range for the DICs
const Vmin = -60 
const Vmax = 0
const V    = range(Vmin, stop=Vmax, step=0.01)

# Definition of the number of cells in the random set
const ncells = 200

# Modifying backend GR attributes
gr(guidefontsize=18, legendfontsize=12, margin=5Plots.mm, grid=false)
myApple = RGBA(187/255, 206/255, 131/255, 1)
mySalmon = RGBA(243/255, 124/255, 130/255)
myYellow = RGBA(228/255, 205/255, 121/255, 1)
myBlue = RGBA(131/255, 174/255, 218/255, 1)
myDarkBlue = RGBA(114/255, 119/255, 217/255, 1)
myOrange = RGBA(241/255, 175/255, 113/255, 1)
myPink = RGBA(243/255, 124/255, 130/255, 1)
myPurple = RGBA(169/255, 90/255, 179/255, 1)
myGreen = RGBA(132/255, 195/255, 168/255, 1)
myRed = RGBA(158/255, 3/255, 8/255, 1)
myGray = RGBA(150/255, 150/255, 150/255, 1)
myLightBlue = RGBA(127/255, 154/255, 209/255, 1)
default(fmt = :png)

# Moving average function
moving_average(vs, n, padding) = [sum(vs[i:(i+n-1)])/n for i in 1:padding:(length(vs)-(n-1))];

# **Plotting FigS2 (uncomment to run, might take a while)**

In [ ]:
# Fixing random seed
Random.seed!(544)

# Initial firing pattern
guth = 4.
Vth = -50.
(g_all_init, ICs_th_init) = degeneracy_fixDICs_neuromod(ncells, 5., guth, Vth);
# create a spiking set with max variability in gCaS and gA

In [ ]:
# Definition of parameters
α = 5e-3 # Rate of transfer between intracellular and membrane
β = 5e-3 # Rate of degradation of intracellular proteins
Kp = 3e-4 # Proprtional gain
Ki = 5e-6 # Integral gain
Kt = β / Ki # Anti-windup gain
gsth_sim(t) = 5 - 13 * pulse(t, Tfinal/2, Tfinal) # Reference
guth_sim(t) = 4.
u_maxCaS = 1e7 # Maximum value of actuator
u_maxA = 1e7

# Definition of homeostatic parameters
tau_g = 100 # Conductance time constant
tau_Na = 600 # Sodium integral action time constants
Ca_tgt(t) = 125. # Calcium target

# Input current definition
Iapp = 0.

tt = 0. : 0.2 : Tfinal

window_size_s = 5
padding_s = 0.2
window_size_ = Int64(round(window_size_s*1000*length(tt)/(Tfinal)))
padding_ = Int64(round(padding_s*1000*length(tt)/(Tfinal)))
tt_moving_average_ = window_size_s/2 : padding_s : Tfinal/1000 - window_size_s/2
tt_moving_average_plot = range(0, Tfinal/1e3, length(tt_moving_average_));

In [ ]:
# gNa_matrix, gCaT_matrix, gCaS_matrix, gA_matrix, gKCa_matrix, gKd_matrix, gH_matrix, 
# gleak_matrix, Ca_ma_matrix = simulate_STG_population(g_all_init, Iapp, tau_Na, tau_g, Ca_tgt,
#                                                      C, α, β, Kp, Ki, Kt, gsth_sim, guth_sim,
#                                                      ICs_th_init, u_maxCaS, u_maxA, tt)

In [ ]:
# writedlm("./data/gNa_matrix.dat", gNa_matrix)
# writedlm("./data/gCaT_matrix.dat", gCaT_matrix)
# writedlm("./data/gCaS_matrix.dat", gCaS_matrix)
# writedlm("./data/gA_matrix.dat", gA_matrix)
# writedlm("./data/gKCa_matrix.dat", gKCa_matrix)
# writedlm("./data/gKd_matrix.dat", gKd_matrix)
# writedlm("./data/gH_matrix.dat", gH_matrix)
# writedlm("./data/gleak_matrix.dat", gleak_matrix)
# writedlm("./data/Ca_ma_matrix.dat", Ca_ma_matrix)

In [ ]:
gNa_matrix = readdlm("./data/gNa_matrix.dat")
gCaT_matrix = readdlm("./data/gCaT_matrix.dat")
gCaS_matrix = readdlm("./data/gCaS_matrix.dat")
gA_matrix = readdlm("./data/gA_matrix.dat")
gKCa_matrix = readdlm("./data/gKCa_matrix.dat")
gKd_matrix = readdlm("./data/gKd_matrix.dat")
gH_matrix = readdlm("./data/gH_matrix.dat")
gleak_matrix = readdlm("./data/gleak_matrix.dat")
Ca_ma_matrix = readdlm("./data/Ca_ma_matrix.dat");

In [ ]:
Random.seed!(22)
# Extracting the maximal ion channel conductances
rng_i1 = mod(rand(Int, 1)[1], ncells) + 1

Random.seed!(2025)
# Extracting the maximal ion channel conductances
rng_i2 = mod(rand(Int, 1)[1], ncells) + 1

Random.seed!(2024)
# Extracting the maximal ion channel conductances
rng_i3 = mod(rand(Int, 1)[1], ncells) + 1;

# Neuron #1

In [ ]:
p1 = plot(ylims=(1e-1, 1e5), yticks=([1e-1, 1e1, 1e3, 1e5], [L"10^{-1}", L"10^{1}", L"10^{3}", L"10^{5}"]), 
          guidefontsize=18, xticks=([0, 100, 200, 300], [L"0", L"100", L"200", L"300"]), tickfontsize=15, 
          size=(600, 300), xlims=(0, 300))

xlabel!(L"t\,\mathrm{(s)}")
ylabel!(L"\bar{g}\,\mathrm{(mS/cm^2)}")

plot!(tt[2:5000:end]./1e3, gNa_matrix[rng_i1, :], color=myApple, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gCaT_matrix[rng_i1, :], color=myYellow, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gA_matrix[rng_i1, :], color=myPurple, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gCaS_matrix[rng_i1, :], color=myGreen, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gKCa_matrix[rng_i1, :], color=myBlue, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gKd_matrix[rng_i1, :], color=myPink, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gH_matrix[rng_i1, :], color=myOrange, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gleak_matrix[rng_i1, :], color=myDarkBlue, linewidth=2.5,
      legend=false, yaxis=:log)

display(p1)
# savefig(p1, "./figures/g_all_crash_1.pdf")

In [ ]:
p9 = plot(legend=false, tickfontsize=15, markerstrokewidth=0., size=(600, 400), 
          xticks=([0, 400, 800, 1200], [L"0", L"400", L"800", L"1200"]), 
          yticks=([0, 40, 80, 120], [L"0", L"40", L"80", L"120"]))

# Interpolating
interpA = linear_interpolation(tt[2:5000:end]./1e3, gA_matrix[rng_i1, :], extrapolation_bc=Line())
interpCaS = linear_interpolation(tt[2:5000:end]./1e3, gCaS_matrix[rng_i1, :], extrapolation_bc=Line())

# Storing
gA_plot_Ca = interpA(tt_moving_average_plot)
gCaS_plot_Ca = interpCaS(tt_moving_average_plot)

# Plotting
t_start=11
i_nmodtt = 4
plot!(gA_plot_Ca[t_start:end], gCaS_plot_Ca[t_start:end], label="", line_z=Ca_ma_matrix[rng_i1, t_start:end], grid=false, c=:thermal,
      linewidth=2, clim=(0, 500), yguidefontsize=18, xguidefontsize=18, alpha=1)


scatter!([gA_plot_Ca[t_start]], [gCaS_plot_Ca[t_start]], color=:white, alpha=0.5, markersize=7)
scatter!([gA_matrix[rng_i1, Int(round(end/2))]], [gCaS_matrix[rng_i1, Int(round(end/2))]], 
        color=myGreen, alpha=0.5, marker=:utriangle, markersize=7)
scatter!([gA_matrix[rng_i1, Int(end/2)+i_nmodtt]], [gCaS_matrix[rng_i1, Int(end/2)+i_nmodtt]], 
        color=myDarkBlue, alpha=0.5, marker=:diamond, markersize=7)
scatter!([gA_matrix[rng_i1, end]], [gCaS_matrix[rng_i1, end]], 
        color=myRed, alpha=0.5, marker=:star5, markersize=7)


xlims!((0, 1200))
ylims!((0, 120))

ylabel!(L"\bar{g}_\mathrm{CaS}\,\mathrm{(mS/cm^2)}")
xlabel!(L"\bar{g}_\mathrm{A}\,\mathrm{(mS/cm^2)}")

display(p9)
# savefig(p9, "./figures/gCaSgA_1.pdf")

# Neuron #2

In [ ]:
p1 = plot(ylims=(1e-1, 1e5), yticks=([1e-1, 1e1, 1e3, 1e5], [L"10^{-1}", L"10^{1}", L"10^{3}", L"10^{5}"]), 
          guidefontsize=18, xticks=([0, 100, 200, 300], [L"0", L"100", L"200", L"300"]), tickfontsize=15, 
          size=(600, 300), xlims=(0, 300))

xlabel!(L"t\,\mathrm{(s)}")
ylabel!(L"\bar{g}\,\mathrm{(mS/cm^2)}")

plot!(tt[2:5000:end]./1e3, gNa_matrix[rng_i2, :], color=myApple, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gCaT_matrix[rng_i2, :], color=myYellow, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gA_matrix[rng_i2, :], color=myPurple, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gCaS_matrix[rng_i2, :], color=myGreen, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gKCa_matrix[rng_i2, :], color=myBlue, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gKd_matrix[rng_i2, :], color=myPink, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gH_matrix[rng_i2, :], color=myOrange, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gleak_matrix[rng_i2, :], color=myDarkBlue, linewidth=2.5,
      legend=false, yaxis=:log)

display(p1)
# savefig(p1, "./figures/g_all_crash_2.pdf")

In [ ]:
p9 = plot(legend=false, tickfontsize=15, markerstrokewidth=0., size=(600, 400), 
          xticks=([0, 400, 800, 1200], [L"0", L"400", L"800", L"1200"]), 
          yticks=([0, 40, 80, 120], [L"0", L"40", L"80", L"120"]))

# Interpolating
interpA = linear_interpolation(tt[2:5000:end]./1e3, gA_matrix[rng_i2, :], extrapolation_bc=Line())
interpCaS = linear_interpolation(tt[2:5000:end]./1e3, gCaS_matrix[rng_i2, :], extrapolation_bc=Line())

# Storing
gA_plot_Ca = interpA(tt_moving_average_plot)
gCaS_plot_Ca = interpCaS(tt_moving_average_plot)

# Plotting
t_start=11
i_nmodtt = 4
plot!(gA_plot_Ca[t_start:end], gCaS_plot_Ca[t_start:end], label="", line_z=Ca_ma_matrix[rng_i2, t_start:end], grid=false, c=:thermal,
      linewidth=2, clim=(0, 500), yguidefontsize=18, xguidefontsize=18, alpha=1)


scatter!([gA_plot_Ca[t_start]], [gCaS_plot_Ca[t_start]], color=:white, alpha=0.5, markersize=7)
scatter!([gA_matrix[rng_i2, Int(round(end/2))]], [gCaS_matrix[rng_i2, Int(round(end/2))]], 
        color=myGreen, alpha=0.5, marker=:utriangle, markersize=7)
scatter!([gA_matrix[rng_i2, Int(end/2)+i_nmodtt]], [gCaS_matrix[rng_i2, Int(end/2)+i_nmodtt]], 
        color=myDarkBlue, alpha=0.5, marker=:diamond, markersize=7)
scatter!([gA_matrix[rng_i2, end]], [gCaS_matrix[rng_i2, end]], 
        color=myRed, alpha=0.5, marker=:star5, markersize=7)


xlims!((0, 1200))
ylims!((0, 120))

ylabel!(L"\bar{g}_\mathrm{CaS}\,\mathrm{(mS/cm^2)}")
xlabel!(L"\bar{g}_\mathrm{A}\,\mathrm{(mS/cm^2)}")

display(p9)
# savefig(p9, "./figures/gCaSgA_2.pdf")

# Neuron #3

In [ ]:
p1 = plot(ylims=(1e-1, 1e5), yticks=([1e-1, 1e1, 1e3, 1e5], [L"10^{-1}", L"10^{1}", L"10^{3}", L"10^{5}"]), 
          guidefontsize=18, xticks=([0, 100, 200, 300], [L"0", L"100", L"200", L"300"]), tickfontsize=15, 
          size=(600, 300), xlims=(0, 300))

xlabel!(L"t\,\mathrm{(s)}")
ylabel!(L"\bar{g}\,\mathrm{(mS/cm^2)}")

plot!(tt[2:5000:end]./1e3, gNa_matrix[rng_i3, :], color=myApple, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gCaT_matrix[rng_i3, :], color=myYellow, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gA_matrix[rng_i3, :], color=myPurple, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gCaS_matrix[rng_i3, :], color=myGreen, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gKCa_matrix[rng_i3, :], color=myBlue, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gKd_matrix[rng_i3, :], color=myPink, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gH_matrix[rng_i3, :], color=myOrange, linewidth=2.5,
      legend=false, yaxis=:log)
plot!(tt[2:5000:end]./1e3, gleak_matrix[rng_i3, :], color=myDarkBlue, linewidth=2.5,
      legend=false, yaxis=:log)

display(p1)
# savefig(p1, "./figures/g_all_crash_3.pdf")

In [ ]:
p9 = plot(legend=false, tickfontsize=15, markerstrokewidth=0., size=(600, 400), 
          xticks=([0, 400, 800, 1200], [L"0", L"400", L"800", L"1200"]), 
          yticks=([0, 40, 80, 120], [L"0", L"40", L"80", L"120"]))

# Interpolating
interpA = linear_interpolation(tt[2:5000:end]./1e3, gA_matrix[rng_i3, :], extrapolation_bc=Line())
interpCaS = linear_interpolation(tt[2:5000:end]./1e3, gCaS_matrix[rng_i3, :], extrapolation_bc=Line())

# Storing
gA_plot_Ca = interpA(tt_moving_average_plot)
gCaS_plot_Ca = interpCaS(tt_moving_average_plot)

# Plotting
t_start=11
i_nmodtt = 4
plot!(gA_plot_Ca[t_start:end], gCaS_plot_Ca[t_start:end], label="", line_z=Ca_ma_matrix[rng_i3, t_start:end], grid=false, c=:thermal,
      linewidth=2, clim=(0, 500), yguidefontsize=18, xguidefontsize=18, alpha=1)


scatter!([gA_plot_Ca[t_start]], [gCaS_plot_Ca[t_start]], color=:white, alpha=0.5, markersize=7)
scatter!([gA_matrix[rng_i3, Int(round(end/2))]], [gCaS_matrix[rng_i3, Int(round(end/2))]], 
        color=myGreen, alpha=0.5, marker=:utriangle, markersize=7)
scatter!([gA_matrix[rng_i3, Int(end/2)+i_nmodtt]], [gCaS_matrix[rng_i3, Int(end/2)+i_nmodtt]], 
        color=myDarkBlue, alpha=0.5, marker=:diamond, markersize=7)
scatter!([gA_matrix[rng_i3, end]], [gCaS_matrix[rng_i3, end]], 
        color=myRed, alpha=0.5, marker=:star5, markersize=7)


xlims!((0, 1200))
ylims!((0, 120))

ylabel!(L"\bar{g}_\mathrm{CaS}\,\mathrm{(mS/cm^2)}")
xlabel!(L"\bar{g}_\mathrm{A}\,\mathrm{(mS/cm^2)}")

display(p9)
# savefig(p9, "./figures/gCaSgA_3.pdf")